# Classification with Azure Machine Learning *designer*

In the previous lab, you used the automated machine learning feature of Azure Machine Learning to train and deploy a machine learning model for *regression*, in which you predicted a numeric value. In this lab, you'll create a model for *classification*, in which the model predicts the category, or *class*, a given entity belongs to.

While automated machine learning makes it easy to try lots of algorithms and find the best performing model for your data, there may be cases where you want more control over how your data is prepared and used to train a model; so this time you'll use another Azure Machine Learning feature called *designer*, which enables you to define workflow for model training in a visual, drag and drop interface.

## What is Classification?

Classification is a form of supervised machine learning in which a model is trained to fit the *features* of an entity to a *label* that represents a particular class. The label is usually an integer indicator, such as 0, 1, or 2; with each indicator representing a different possible classification. As you might recall from the previous lab, we often think of a machine learning model as a function (***f***) that operates on features (**x**) to predict a label (**y**); and in this case, **y** is a numeric indicator for a class.

$$y = f(x)$$

Let's look at an example to make things a little clearer.


<p style='text-align:center'><img src='./images/diabetes.jpg' alt="Clinical data for multiple patients, some of whom are diabetic and some of whom aren't"/></p>

Suppose a health clinic offers patients a general health screening, where they can provide their personal details (age, number of children, and so on) and have some medical metrics measured (weight, blood pressure, and so on). The screening might also include a test for diabetes, for which some patients may test negative and others positive. So there's a sense in which there are two *classes* of patient; those with diabetes, and those without. We could assign numeric labels to those classes, with ***0*** meaning that the patient tested negative for diabetes, and ***1*** meaning a positive diagnosis.

Now, let's suppose that the diabetes test is expensive to conduct, and stressful for the patient being tested.

> **Tip**: it's not. We're just imagining it is for the purposes of this lab example. If you think you're at risk *please* go and get tested!

The clinic might want to restrict testing to only those patients with a high probability of testing positive. So the challenge is to take all of the other information we have about the patients (age, weight, blood pressure, and so on) and try to find a correlation with the classification of the patient as non-diabetic or diabetic. In other words, we use the patient's medical measurements as *features* to predict a class *label* that indicates the likelihood of a positive diagnosis for diabetes.

> **A slight technicality (which will be important later!)**
>
> Up to this point, we've throught of a classification model as predicting a numeric class indicator, like 0 or 1. Actually, it's a little more complicated than that.
>
> What the model actually calculates is the *probability* of the entity belonging to each possible label - so the result of the function (*f*) is actually a vector (in other words, an array of values) that contains a probability score for each possible class.
>
> For our diabetes example, the function might return a vector such as $[0.2, 0.8]$, which indicates that there's a 0.2 (20%) probability that this particular patient belongs to class 0 (non-diabetic), and 0.8 (80%) probability that they belong to class 1 (diabetic). We classify the entity based on the most probable label prediction, so in this case the patient would be classified as class 1 (diabetic).
> 
> Note that the individual class probabilities always add up to 1 - there's a 100% probability that the patient is either diabetic or non-diabetic!
> 
> In this case, the model we're creating only has two possible classes - we call this *binary* classification, and you can simplify things by considering only the probability of the *positive* class (in this case, *diabetic*) - for example, you might describe a probability prediction as 80%, by which you mean there's an 80% probability of the patient being diabetic (and therefore a 20% probability that they're not). A probability prediction of 30% on the other hand means there's a 70% probability that the *negative* class is correct (in other words, the patient is not diabetic).
>
> It's also possible to have classification scenarios where there are multiple possible classes. Unsuprisingly, we call this *multiclass-classification*. For example, we could create a model to predict non-diabetic, type A diabetes, and type B diabetes classes. This would result in a vector of three probability values (one for each possible class), which would still add up to 1.

## Prepare Compute and Data for Model Training

In the previous lab, you created an Azure Machine Learning workspace that contains a training cluster compute resource.

> **If you didn't complete the previous lab, or you've deleted your workspace, follow these steps to get caught up:**
>
> 1. Sign into the [Azure portal](https://portal.azure.com) using your Microsoft credentials.
> 2. Click the **&#65291;Create a resource** button, search for *Machine Learning*, and create a new **Machine Learning** resource the following settings:
>     - **Workspace Name**: *A unique name of your choice*
>     - **Subscription**: *Your Azure subscription*
>     - **Resource group**: *Create a new resource group with a unique name*
>     - **Location**: *Choose any available location*
>     - **Workspace edition**: Enterprise
> 3. Wait for your workspace to be created. Then go to it in the portal.
> 4. In the Azure portal, in the **Overview** page for your Azure Machine Learning workspace, click the link to launch Azure Machine Learning **studio**. Alteratively, browse to [https://ml.azure.com](https://ml.azure.com]), sign in using your Microsoft credentials, and select your Azure subscription and Azure Machine Learning workspace.
> 5. View the **Compute** page (under **manage**).
> 6. Switch to the **Training Clusters** tab, and add a new training cluster with the following settings:
>     - **Compute name**: aml-cluster
>     - **Virtual Machine size**: Standard_DS1_v2
>     - **Virtual Machine priority**: Dedicated
>     - **Minimum number of nodes**: 2
>     - **Maximum number of nodes**: 2
>     - **Idle seconds before scale down**: 3600

### Prepare Compute Resources

In this lab, you'll use the *training cluster* you created previously to train your model. You'll also use an *inference cluster* to host the trained model as a service that applications can use.

1. In [Azure Machine Learning studio](https://ml.azure.com), on the **Compute** page for your workspace, on the **Inference Clusters** tab, add a new cluster with the following settings:
    * **Compute name**: aks-cluster
    * **Kubernetes Service**: Create new
    * **Region**: *Any available region*
    * **Virtual Machine size**: Standard_DS2_v2 (*Use the filter to find this in the list*)
    * **Cluster purpose**: Dev-test
    * **Number of nodes**: 2
    * **Network configuration**: Basic
    * **Enable SSL configuration**: Unselected
2. Verify that the compute target is in the *Creating* state - it will take a while to be created, so leave it for now.
3. On the **Compute** page, on the **Training clusters** tab, open the **aml-cluster** compute target you created previously and click **Edit**. Then set the **Minimum number of nodes** setting to **1** and click **Update**.

### Prepare Data

Now that you have some compute resources that you can use to process data, you'll need a way to store and ingest the data to be processed.

1. In [Azure Machine Learning studio](https://ml.azure.com), view the **Datasets** page. Datasets represent specific data files or tables that you plan to work with in Azure ML.
2. Create a new dataset from web files, using the following settings:
    * **Basic Info**:
        * **Web URL**: https://aka.ms/diabetes-data
        * **Name**: diabetes dataset
        * **Dataset type**: Tabular
        * **Description**: Diabetes data
    * **Settings and preview**:
        * **File format**: Delimited
        * **Delimiter**: Comma
        * **Encoding**: UTF-8
        * **Column headers**: Use headers from first file
        * **Skip rows**: None
    * **Schema**:
        * Include all columns other than **Path**
        * Review the automatically detected types
    * **Confirm details**:
        * Do not profile the dataset after creation
3. After the dataset has been created, open it and view the **Explore** page to see a sample of the data. This data represents details from patients who have been tested for diabetes.

## Create a Designer Pipeline and Explore Data

To get started with Azure machine Learning designer, first you must create a pipeline and add the dataset you want to work with.

1. In [Azure Machine Learning studio](https://ml.azure.com) for your workspace, view the **Designer** page and create a new pipeline.
2. In the **Settings** pane, change the default pipeline name (**Pipeline-Created-on-*date***) to **Visual Diabetes Training** (if the **Settings** pane is not visible, click the **&#9881;** icon next to the pipeline name at the top).
3. Note that you need to specify a compute target on which to run the pipeline. In the **Settings** pane, click **Select compute target** and select the **aml-cluster** compute target you created in the previous lab.
4. On the left side of the designer, expand the **Datasets** section, and drag the **diabetes dataset** dataset you created in the previous exercise onto the canvas.
5. Select the **diabetes dataset** module on the canvas, and view its settings. Then on the **outputs** tab, click the **Visualize** icon (which looks like a column chart).
6. Review the schema of the data, noting that you can see the distributions of the various columns as histograms. Then close the visualization.

## Add Transformations

Before you can train a model, you typically need to apply some preprocessing transformations to the data.

1. In the pane on the left, expand the **Data Transformation** section, which contains a wide range of modules you can use to transform data and pre-process it before model training. Drag a **Normalize Data** module to the canvas, below the **diabetes dataset** module. Then connect the output from the **diabetes dataset** module to the input of the **Normalize Data** module.
2. Select the **Normalize Data** module and view its settings, noting that it requires you to specify the transformation method and the columns to be transformed. Then, leaving the transformation as **ZScore**, edit the columns to includes the following column names:
    * PlasmaGlucose
    * DiastolicBloodPressure
    * TricepsThickness
    * SerumInsulin
    * BMI
    * DiabetesPedigree

    **Note**: We're normalizing the numeric columns to put them on the same scale, and avoid columns with large values doiminating model training. You'd usually apply a whole bunch of pre-processing transformations like this to prepare your data for training, but we'll keep things simple in this exercise.

3. Now we're ready to split the data into separate datasets for training and validation. This enables you to train the model using some of the data, but hold back some data that you can use to test the trained model. In the pane on the left, in the **Data Transformations** section, drag a **Split Data** module onto the canvas under the **Normalize Data** module. Then connect the *Transformed Dataset* (left) output of the **Normalize Data** module to the input of the **Split Data** module.
4. Select the **Split Data** module, and configure its settings as follows:
    * **Splitting mode** Split Rows
    * **Fraction of rows in the first output dataset**: 0.7
    * **Random seed**: 123
    * **Stratified split**: False

## Add Model Training Modules

With the data prepared and split into training and validation datasets, you're ready to configure the pipeline to train and evaluate a model.

1. Expand the **Model Training** section in the pane on the left, and drag a **Train Model** module to the canvas, under the **Split Data** module. Then connect the *Result dataset1* (left) output of the **Split Data** module to the *Dataset* (right) input of the **Train Model** module.
2. The model we're training will predict the **Diabetic** value, so select the **Train Model** module and modify its settings to set the **Label column** to  **Diabetic** (matching the case and spelling exactly!)
3. The **Diabetic** label the model will predict is a binary column (1 for patients who have diabetes, 0 for patients who don't), so we need to train the model using a *classification* algorithm. Expand the **Machine Learning Algorithms** section, and under **Classification**, drag a **Two-Class Logistic Regression** module to the canvas, to the left of the **Split Data** module and above the **Train Model** module. Then connect its output to the **Untrained model** (left) input of the **Train Model** module.
4. To test the trained model, we need to use it to *score* the validation dataset we held back when we split the original data - in other words, predict labels for the features in the validation dataset. Expand the **Model Scoring & Evaluation** section and drag a **Score Model** module to the canvas, below the **Train Model** module. Then connect the output of the **Train Model** module to the **Trained model** (left) input of the **Score Model** module; and drag the **Results dataset2** (right) output of the **Split Data** module to the **Dataset** (right) input of the **Score Model** module.
5. To evaluate how well the model performs, we need to look at some metrics generated by scoring the validation dataset. From the **Model Scoring & Evaluation** section, drag an **Evaluate Model** module to the canvas, under the **Score Model** module, and connect the output of the **Score Model** module to the **Score dataset** (left) input of the **Evaluate Model** module.

## Run the Training Pipeline

With the data flow steps defined, you're now ready to run the training pipeline and train the model.

1. Verify that your pipeline looks similar to the following (note that the image includes comments in each module to document what they're doing - it's not a bad idea to do this when you're using the Designer for a real project!):

<p style='text-align:center'><img src='./images/visual-training.jpg' alt="A visual training pipeline"/></p>

2. At the top right, click **Run**. Then when prompted, create a new *experiment* named **visual-training**, and run it.  This will initialize the compute target and then run the pipeline, which may take 10 minutes or longer. You  can see the status of the pipeline run above the top right of the design canvas.

    **Tip**: While it's running, you can view the pipeline and experiment that have been created in the **Pipelines** and **Experiments** pages. Switch back to the **Visual Diabetes Training** pipeline on the **Designer** page when you're done.

3. After the **Normalize Data** module has finished (indicated by a &#x2705; icon), select it, and in the **Settings** pane, on the **Outputs + Logs** tab, under **Port outputs** in the **Transformed dataset** section, click the **Visualize** icon, and note that you can view statistics and distribution visualizations for the transformed columns.
4. Close the **Normalize Data** visualizations, close or resize the settings pane (click the X or **<sub>&#8599;</sub><sup>&#8601;</sup>** icon), and wait for the rest of the modules to complete.

## Evaluate the Model

The validation data you witheld and used to score the model includes the known values for the label; so to validate the model, you can compare the true values for the label to the label values that were predicted when you scored the validation dataset. Based on this comparison, you can calculate various metrics that describe how well the model performs.

1. Visualize the output of the **Evaluate Model** module to see the performance metrics for the model. There's a lot of information here that can help data scientists assess how well the model predicts based on the validation data.
2. View the *confusion matrix* for the model. This is a tabulation of the predicted and actual value counts for each possible class. For a binary classification model like this one, where you are predicting one of two possible values, the confusion matrix is a 2x2 grid showing the predicted and actual value counts for classes **0** and **1**, similar to this:

<p style='text-align:center'><img src='./images/confusion_matrix.jpg' alt="A confusion matrix showing actual and predicted value counts for each class"/></p>

The confusion matrix shows the counts in a particular format so that cases where both the predicted value and the actual value was 1 (known as *true positives*) are at the top left, and cases where both the predicted value and the actual value was 0 (*true negatives*) are at the bottom right. The other cells show cases where the predicted and actual values differ (*false positives* and *false negatives*). The cells in the matrix are colored so that the more cases represented in the cell, the more intense the color - with the result that you can identify a model that predicts accurately for all classes by looking for a diagonal line of intensely colored cells from the top left to the bottom right (in other words, the cells where the predicted values match the actual values). For a multi-class classification model (where there are more than two possible classes), the same approach is used to tabulate each possible combination of actual and predicted value counts - so a model with three possible classes would result in a 3x3 matrix with a diagonal line of cells where the predicted and actual labels match.

3. Review the metrics to the left of the confusion matrix, which include:
    - **Accuracy**: The ratio of correct predictions (true positives + true negatives) to the total number of predictions. In other words, what proportion of diabetes predictions did the model get right?
    - **Precision**: The fraction of positive cases correctly identified (the number of true positives divided by the number of true positives plus false positives). In other words, out of all the patients that the model predicted as having diabetes, how many are actually diabetic?
    - **Recall**: The fraction of the cases classified as positive that are actually positive (the number of true positives divided by the number of true positives plus false negatives). In other words, out of all the patients who actually have diabetes, how many did the model identify?
    - **F1 Score**: An overall metric that essentially combines precision and recall.
    - *We'll return to **AUC** later*.

Of these metric, *accuracy* is the most intuitive. However, you need to be careful about using simple accuracy as a measurement of how well a model works. Suppose that only 3% of the population is diabetic. You could create a model that always predicts **0** and it would be 97% accurate - just not very useful! For this reason, most data scientists use other metrics like precision and recall to assess classification model performance.

4. Above the list of metrics, note that there's a **Threshold** slider. Remember that what a classification model predicts is the probability for each possible class. In the case of this binary classification model, the predicted probability for a *positive* (i.e. diabetic) prediction is a value between 0 and 1. By default, a predicted probability for diabetes above 0.5 results in a class prediction of 1, while a prediction below this threshold means that there's a greater probability of the patient **not** having diabetes (remember that the probabilities for all classes add up to 1), so the predicted class would be 0. Try moving the threshold slider and observe the effect on the confusion matrix. If you move it all the way to the left (0), the Recall metric becomes 1, and if you move it all the way to the right (1), the Recall metric becomes 0.

5. Look above the Threshold slider at the **ROC curve** (ROC stands for *received operator characteristic*, but most data scientists just call it a ROC curve). Another term for *recall* is **True positive rate**, and it has a corresponding metric named **False positive rate** which measures the number of negative cases incorrectly identified positives compared the number of actual negative cases. Plotting these metrics against each other for every possible threshold value between 0 and 1 results in a curve. In an ideal model, the curve would go all the way up the left side and across the top, so that it covers the full area of the chart. The larger the *area under the curve* (which can be any value from 0 to 1), the better the model is performing - this is the **AUC** metric listed with the other metrics below. To get an idea of how this area represents the performance of the model, imagine a straight diagonal line from the bottom left to the top right of the ROC chart. This represents the expected performance if you just guessed or flipped a coin for each patient - all things being equal, you could expect to get around half of them right, and half of them wrong, so the area under the diagonal line represents an AUC of 0.5. If the AUC for your model is higher than this for a binary classification model, then the model performs better than a random guess.

**Note**: The performance of this model isn't all that great, partly because we performed only minimal feature engineering and pre-processing. You could try some different classification algorithms and compare the results (you can connect the outputs of the **Split Data** module to multiple **Train Model** and **Score Model** modules, and you can connect a second scored model to the **Evaluate Model** module to see a side-by-side comparison). The point of the exercise is simply to introduce you to classification and the Azure Machine Learning designer interface, not to train a perfect model!

## Create an Inference Pipeline

Now you have a pipeline to train the model, you need a second pipeline that performs the same data transformations for new data, and then uses the trained model to *inference* (in other words, predict) label values based on its features. This will form the basis for a predictive service that you can publish for applications to use.

1. View the **Visual Diabetes Training** pipeline you created in the previous exercise.
2. In the **Create inference pipeline** drop-down list, click **Real-time inference pipeline**. After a few seconds, a new version of your pipeline named **Visual Diabetes Training-real time inference** will be opened.
3. Rename the new pipeline to **Predict Diabetes**, and then review the new pipeline. Note that some of the transformations and training steps have been encapsulated in this pipeline so that the statistics from your training data will be used to normalize any new data values, and the trained model will be used to score the new data.
4. The inference pipeline assumes that new data will match the schema of the original training data, so the **diabetes dataset** module from the training pipeline is included. However, this input data includes the **Diabetic** label that the model predicts, which is unintuitive to include in new patient data for which a diabetes prediction has not yet been made. Delete this module and replace it with an **Enter Data Manually** module from the **Data Input and Output** section, connected to the same **dataset** input of the **Apply Transformation** module as the **Web Service Input**. Then modify the settings of the **Enter Data Manually** module to use the following CSV input, which includes feature values without labels for three new patient observations:

    ```CSV
    PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age
    1882185,9,104,51,7,24,27.36983156,1.350472047,43
    1662484,6,73,61,35,24,18.74367404,1.074147566,75
    1228510,4,115,50,29,243,34.69215364,0.741159926,59
    ```

5. The inference pipeline includes the **Evaluate Model** module, which is not useful when predicting from new data, so delete this module.
6. The output from the **Score Model** module includes all of the input features as well as the predicted label and probability score. To limit the output to only the prediction and probability, delete the connection between the **Score Model** module and the **Web Service Output**, add an **Execute Python Script** module from the **Python Language** section, connect the output from the **Score Model** module to the **Dataset1** (left-most) input of the **Execute Python Script**, and connect the output of the **Execute Python Script** module to the **Web Service Output**. Then modify the settings of the **Execute Python Script** module to use the following code (replacing all existing code):

    ```Python
    import pandas as pd

    def azureml_main(dataframe1 = None, dataframe2 = None):

        scored_results = dataframe1[['PatientID', 'Scored Labels', 'Scored Probabilities']]
        scored_results.rename(columns={'Scored Labels':'DiabetesPrediction',
                                       'Scored Probabilities':'Probability'},
                              inplace=True)
        return scored_results


    ```

7. Verify that your pipeline looks similar to the following:

<p style='text-align:center'><img src='./images/visual-inference.jpg' alt="A visual inference pipeline"/></p>

8. Run the pipeline as a new experiment named **predict-diabetes** on the **aml-compute** compute target you used for training. This may take a while!

## Publish an Inferencing Service

Now you have an inference pipeline for real-time inferencing, which you can publish as a service for client applications to use.

1. Return the the **Compute** page and on the **Inference Compute** tab, refresh the view and verify that your **aks-cluster** compute has been created. If not, wait for your inference cluster to be created. This may take quite a bit of time.
2. Switch back to the **Designer** tab and reopen your **Predict Diabetes** inference pipeline. If it has not yet finished running, await it's completion. Then visualize the output of the **Execute Python Script** module to see the predicted labels and probabilties for the three patient observations in the input data.
3. At the top right, click **Deploy**, and set up a new real-time endpoint named **predict-diabetes** on the **aks-cluster** compute target you created.
4. Wait for the web service to be deployed - this can take several minutes. The deployment status is shown at the top left of the Designer interface.

## Test the Service

Now you can test your deployed service from a client application - in this case, you'll use the code in the cell below to simulate a client application.

1. On the **Endpoints** page, open the **predict-diabetes** real-time endpoint.
2. When the **predict-diabetes** endpoint opens, on the **Consume** tab, note the **REST endpoint** and **Primary key** values.
3. In the cell below, replace the **ENDPOINT** and **PRIMARY_KEY** placeholders with the values for your service, which you can copy from the **Consume** tab on the page for your endpoint.
4. Run the code cell and view the output returned by your web service.

> **Note** Don't worry about the details of the code - we're simply verifying that your published service works!

In [ ]:
endpoint = 'YOUR_ENDPOINT' #Replace with your endpoint
key = 'YOUR_KEY' #Replace with your key

import urllib.request
import json
import os

data = {
    "Inputs": {
        "WebServiceInput0":
        [
            {
                    'PatientID': 1882185,
                    'Pregnancies': 9,
                    'PlasmaGlucose': 104,
                    'DiastolicBloodPressure': 51,
                    'TricepsThickness': 7,
                    'SerumInsulin': 24,
                    'BMI': 27.36983156,
                    'DiabetesPedigree': 1.3504720469999998,
                    'Age': 43,
            },
        ],
    },
    "GlobalParameters":  {
    }
}

body = str.encode(json.dumps(data))
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ key)}
req = urllib.request.Request(endpoint, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    json_result = json.loads(result)
    output = json_result["Results"]["WebServiceOutput0"][0]
    print('Patient: {}\nPrediction: {}\nProbability: {:.2f}'.format(output["PatientID"],
                                                            output["DiabetesPrediction"],
                                                            output["Probability"],))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers to help debug
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

## Reset Resources

The web service is hosted in a Kubernetes cluster. If you don't intend to experiment with it further, you should delete the endpoint and the cluster to avoid accruing unnecessary Azure charges. You should also stop other compute resources until you need them again.

1. In [Azure Machine Learning studio](https://ml.azure.com), on the **Endpoints** tab, select the **predict-diabetes** endpoint. Then click the **Delete** (&#128465;) button and confirm that you want to delete the endpoint.
2. On the **Compute** page, on the **Inference Clusters** tab, select the select the **aks-cluster** endpoint. Then click the **Delete** (&#128465;) button and confirm that you want to delete the compute target.
3. On the **Compute** page, on the **Training clusters** tab, open the **aml-cluster** compute target and click **Edit**. Then set the **Minimum number of nodes** setting to **0** and click **Update**.